In [3]:
import numpy as np
import os
import pandas as pd
import requests
import sqlite3
import re

from bs4 import BeautifulSoup

# Get Event Links

* Establish connection to sqlite database file
* Get event links from db file
* Store in a dataframe

In [9]:
os.getcwd()

'/Users/gregmartinez/projects/mtg_scraper'

In [5]:
db = "dbs/links.db"
conn = sqlite3.connect(db)

event_df = pd.read_sql("SELECT * FROM event", conn)

conn.close()

In [6]:
# Cleaning
event_df["date"] = pd.to_datetime(event_df["date"], format="%d/%m/%y")

In [7]:
event_df.sort_values(by="date", ascending=False)

,id,name,link,date
363,364,Japan Championship 2020 Winter Daily Trial @ B...,https://www.mtgtop8.com/event?e=28293&f=ST,2020-11-26
362,363,Keep it Simple Stupid @ Magic for Old Guys,https://www.mtgtop8.com/event?e=28288&f=ST,2020-11-25
361,362,Free Daily @ Owl Central Games,https://www.mtgtop8.com/event?e=28287&f=ST,2020-11-25
360,361,Japan Championship 2020 Winter Daily Trial @ B...,https://www.mtgtop8.com/event?e=28286&f=ST,2020-11-25
325,326,Daily @ Dablacksplash Convergence,https://www.mtgtop8.com/event?e=28277&f=ST,2020-11-24
...,...,...,...,...
228,229,Free FNM @ Equalssport,https://www.mtgtop8.com/event?e=27190&f=ST,2020-09-04
229,230,Dealers at Home! @ MagicDealers,https://www.mtgtop8.com/event?e=27193&f=ST,2020-09-04
230,231,FNM @ Lotus eSports,https://www.mtgtop8.com/event?e=27194&f=ST,2020-09-04
231,232,SCG Tour Online - Challenge @ Star City Games,https://www.mtgtop8.com/event?e=27172&f=ST,2020-09-03


In [9]:
events = list(event_df.sort_values("date", ascending=False)["link"])
events

['https://www.mtgtop8.com/event?e=28293&f=ST',
 'https://www.mtgtop8.com/event?e=28288&f=ST',
 'https://www.mtgtop8.com/event?e=28287&f=ST',
 'https://www.mtgtop8.com/event?e=28286&f=ST',
 'https://www.mtgtop8.com/event?e=28277&f=ST',
 'https://www.mtgtop8.com/event?e=28265&f=ST',
 'https://www.mtgtop8.com/event?e=28266&f=ST',
 'https://www.mtgtop8.com/event?e=28269&f=ST',
 'https://www.mtgtop8.com/event?e=28273&f=ST',
 'https://www.mtgtop8.com/event?e=28274&f=ST',
 'https://www.mtgtop8.com/event?e=28267&f=ST',
 'https://www.mtgtop8.com/event?e=28268&f=ST',
 'https://www.mtgtop8.com/event?e=28292&f=ST',
 'https://www.mtgtop8.com/event?e=28262&f=ST',
 'https://www.mtgtop8.com/event?e=28245&f=ST',
 'https://www.mtgtop8.com/event?e=28250&f=ST',
 'https://www.mtgtop8.com/event?e=28263&f=ST',
 'https://www.mtgtop8.com/event?e=28239&f=ST',
 'https://www.mtgtop8.com/event?e=28219&f=ST',
 'https://www.mtgtop8.com/event?e=28220&f=ST',
 'https://www.mtgtop8.com/event?e=28232&f=ST',
 'https://www

### Event with Two Players

In [131]:
html = requests.get(events[0])
soup = BeautifulSoup(html.text)

body = soup.body

In [132]:
deck1 = body.find_all("div", class_="W14")
deck1_rank = deck1[0].text
deck1_name = deck1[1].find("a").text

print(deck1_rank, ", ", deck1_name)

deck2 = body.find_all("div", class_="S14")
deck2_rank = deck2[0].text
deck2_name = deck2[1].find("a").text

print(deck2_rank, ", ", deck2_name)
print(len(deck2))

1 ,  Gruul Aggro
2 ,  UW Control
5


In [80]:
players = body.find_all("div", class_="G11")
player1 = players[0].find("a").text
player2 = players[1].find("a").text

print(player1, player2, sep="\n")

Antimo Morlando
Umberto Veneruso


### Event with One Player

In [96]:
html = requests.get(events[1])
soup = BeautifulSoup(html.text)
body = soup.body

In [97]:
deck = body.find_all("div", class_="W14")
deck_rank = deck[0].text
deck_name = deck[1].find("a").text
print(deck_rank, ", ", deck_name)

nothing = body.find_all("div", class_="S14")
print(len(nothing))

1 ,  Red Deck Wins
3


In [98]:
player = body.find_all("div", class_="G11")
print(len(player))
player1 = player[0].find("a").text
player1

1


'Ricardo Azeredo'

### Event with Multiple Players

In [99]:
html = requests.get(events[5])
soup = BeautifulSoup(html.text)
body = soup.body

In [101]:
html.url

'https://www.mtgtop8.com/event?e=27921&f=ST'

In [120]:
deck1 = body.find_all("div", class_="W14")
deck1_rank = deck1[0].text
deck1_name = deck1[1].find("a").text
print(deck1_rank, ", ", deck1_name)

other_decks = body.find_all("div", class_="S14")
ranks = []
deck_names = []
for idx, result in enumerate(other_decks[:-3]):
    if idx % 2 == 0:
        ranks.append(result.text)
    else:
        deck_names.append(result.find("a").text)
print(ranks)
print(deck_names)

1 ,  Gruul Aggro
['2', '3-4', '3-4', '5-8', '5-8', '5-8', '5-8']
['Selesnya Aggro', 'Dimir Rogue', 'Gruul Aggro', 'Dimir Flash', 'Gruul Aggro', 'Gruul Aggro', 'Rakdos Aggro']


In [121]:
players = body.find_all("div", class_="G11")

for player in players:
    print(player.find("a").text)


Junichi Takayama
Yoshiro Mikami
Kazuki Katou
Norikazu Ichikawa
Yasuhiro Shinkai
Drmonday
?? ????
Teruya Kakumae


### Looping through all events for deck name & players

* Winner is W14
* Others are S14; if rank is reported in points, then it uses S12

In [31]:
for event in events[:10]:
    html = requests.get(event)
    soup = BeautifulSoup(html.text)
    body = soup.body
    
    print(html.url)
    try:
        deck1 = body.find_all("div", class_="W14")
        deck1_rank = deck1[0].text
        deck1_name = deck1[1].find("a").text
        deck1_link = deck1[1].find("a").get("href")
    except IndexError:
        deck1_rank = 1
        deck1_name = deck1[0].find("a").text
        deck1_link = deck1[0].find("a").get("href")
        print(deck1_rank, ", ", deck1_name, ", ", deck1_link)
        print("The above was a problem url!")
    print(deck1_rank, ", ", deck1_name, ", ", deck1_link)

https://www.mtgtop8.com/event?e=28287&f=ST
1 ,  Dimir Control ,  ?e=28287&d=423952&f=ST
https://www.mtgtop8.com/event?e=28286&f=ST
1 ,  Naya Aggro ,  ?e=28286&d=423944&f=ST
https://www.mtgtop8.com/event?e=28265&f=ST
1 ,  Mono Green Aggro ,  ?e=28265&d=423809&f=ST
https://www.mtgtop8.com/event?e=28266&f=ST
1 ,  Dimir Control ,  ?e=28266&d=423812&f=ST
https://www.mtgtop8.com/event?e=28269&f=ST
1 ,  Temur Ultimatum ,  ?e=28269&d=423828&f=ST
https://www.mtgtop8.com/event?e=28273&f=ST
1 ,  Dimir Control ,  ?e=28273&d=423864&f=ST
https://www.mtgtop8.com/event?e=28274&f=ST
1 ,  Grixis Control ,  ?e=28274&d=423868&f=ST
https://www.mtgtop8.com/event?e=28277&f=ST
1 ,  Dimir Control ,  ?e=28277&d=423881&f=ST
https://www.mtgtop8.com/event?e=28267&f=ST
1 ,  Gruul Aggro ,  ?e=28267&d=423820&f=ST
https://www.mtgtop8.com/event?e=28268&f=ST
1 ,  Temur Ultimatum ,  ?e=28268&d=423824&f=ST


In [69]:
from src.sqldb import SQLDatabase

def get_deck_table():
    """Opens event table and gets deck data from each link in event table"""
    event_df = _open_sql("event")
    results = []
    for link, event in zip(event_df["link"], event_df["name"]):
        html = requests.get(link)
        soup = BeautifulSoup(html.text, features="lxml")
        body = soup.body

        names, ranks, links = _get_winners(body)

        has_points = False
        if _is_points(ranks[0]):
            has_points = True

        other_names, other_ranks, other_links = _add_other_decks(body, has_points)
        names.extend(other_names)
        ranks.extend(other_ranks)
        links.extend(other_links)

        players = _get_players(body, names)
        

        assert _are_equal_length(names, ranks, players, links), \
            "names: " + str(len(names)) \
            + ", ranks: " + str(len(ranks)) \
            + ", players: " + str(len(players)) \
            + ", links: " + str(len(links)) \
            + " at " + html.url
        assert _are_equal_length(
            list(zip(names, ranks, players, links)), names, ranks, players, links)

        results.extend(
            [
                (event, player, name, rank, link)
                for (player, name, rank, link)
                in zip(players, names, ranks, links)
            ]
        )

    return results

def _open_sql(table):
    with SQLDatabase() as sql_db:
        return sql_db.get_dataframe_from(table)

def _get_winners(body):
    try:
        winner_rank = body.find_all("div", class_="W12")[1].text
        winner_name = body.find_all("div", class_="W14")[0].find("a").text
        winner_link = body.find_all("div", class_="W14")[0].find("a").get("href")
    except (IndexError, AttributeError):
        winner_rank = body.find_all("div", class_="W14")[0].text
        winner_name = body.find_all("div", class_="W14")[1].find("a").text
        winner_link = body.find_all("div", class_="W14")[1].find("a").get("href")
    finally:
        return [winner_name], [winner_rank], [winner_link]


def _add_other_decks(body, has_points=False):
    names = []
    ranks = []
    links = []
    if has_points:
        for name in body.find_all("div", class_="S14")[:-3]:
            assert not _is_malformed(
                name.text), "Bad name - " + name.text
            names.append(name.text.strip())
            links.append(name.find("a").get("href"))
        for points in body.find_all("div", class_="S12"):
            if not _should_be_skipped(points.text):
                assert _is_points(points.text), "Malformed Points " + \
                    points.text
                ranks.append(points.text.strip())
    else:
        for idx, result in enumerate(body.find_all("div", class_="S14")[:-3]):
            if idx % 2 == 0:
                if not _is_rank(result.text):
                    break
                ranks.append(result.text.strip())
            else:
                assert not _is_malformed(
                    result.text), "Bad name - " + result.text
                names.append(result.text.strip())
                links.append(result.find("a").get("href"))
    return names, ranks, links


def _get_players(body, names):
    result = []
    for player in body.find_all("div", class_="G11")[:len(names)]:
        result.append(player.find("a").text)
    return result


def _is_malformed(name):
    return re.match("^\$[0-9]*\ \(.*\)$", name) or re.match("^[0-9]*\ TIX$", name)


def _should_be_skipped(rank):
    return rank == "close" or rank == "Companion card"


def _is_points(rank):
    return re.match("^[0-9]*\ pts$", rank)


def _is_rank(rank):
    return re.match("^[0-9]*-[0-9]*$", rank) or re.match("^[0-9]*$", rank)


def _are_equal_length(*args):
    equal = True
    length1 = len(args[0])
    for arg in args:
        if len(arg) != length1:
            equal = False
    return equal



In [70]:
result = get_deck_table()
    

In [76]:
names = {}
for item in result:
    if item[1] not in names:
        names[item[1]] = 1
    else:
        pass

In [80]:
conn = sqlite3.connect("dbs/links.db")
cursor = conn.cursor()

In [83]:
result[0][0]

'FNM @ MTG Arena Campania'

In [98]:
cursor.execute(f"""
SELECT id
FROM event
WHERE link = ? 
""", ("https://www.mtgtop8.com/event?e=28287&f=ST",)
).fetchone()[0]

362

In [105]:
name_key = result[0][2:]

In [106]:
name_key

('Gruul Aggro', '1', '?e=27938&d=421597&f=ST')

In [120]:
for item in result:
    print(cursor.execute("""
    SELECT id
    FROM pilot
    WHERE (firstName || lastName) = ?
    """, ("".join(item[1].split(maxsplit=1)),)
    ).fetchone()[0])

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
12
24
25
26
27
28
29
30
31
32
33
34
35
36
22
37
38
39
40
41
42
43
44
45
46
47
48
49
50
22
51
52
53
54
55
56
18
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
28
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
59
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
85
191
192
193
194
195
196
197
198
199
200
201
18
202
55
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
239
245
246
247
248
210
249
250
251
252
253
254
255
256
257
258
259
260
261
15
262
30
263
264
265
266
267
268

In [118]:
cursor.execute("""
SELECT *
FROM pilot""").fetchall()

[(3, 'Antimo', 'Morlando'),
 (4, 'Umberto', 'Veneruso'),
 (5, 'Ricardo', 'Azeredo'),
 (6, 'Boris', 'Pan'),
 (7, 'Kazuyuki', 'Takimura'),
 (8, 'Tomohiro', 'Tsuda'),
 (9, 'Takahiro', 'Omiya'),
 (10, 'Tomohiro', 'Mashima'),
 (11, 'Hideki', 'Yakushi'),
 (12, 'Drmonday', ''),
 (13, 'Calm', ''),
 (14, 'Tomonori', 'Hirami'),
 (15, 'Rashid', 'Davis'),
 (16, 'Daniel', 'Alamos'),
 (17, 'Hilário', 'Nunes'),
 (18, 'Abel', 'Jacoby'),
 (19, 'Junichi', 'Takayama'),
 (20, 'Yoshiro', 'Mikami'),
 (21, 'Kazuki', 'Katou'),
 (22, 'Norikazu', 'Ichikawa'),
 (23, 'Yasuhiro', 'Shinkai'),
 (24, '??', '????'),
 (25, 'Teruya', 'Kakumae'),
 (26, 'Matthew', 'Davis'),
 (27, 'Bogdan', 'Simonov'),
 (28, 'Howtoplay', ''),
 (29, 'Mikail', 'Nogueira'),
 (30, 'Masatoshi', 'Ota'),
 (31, 'Santiago', 'Ramirez'),
 (32, 'Uragram', ''),
 (33, 'Ken', 'Sawada'),
 (34, 'Shinnosuke', 'Wanaka'),
 (35, 'Keita', 'Tonouchi'),
 (36, 'Shunichi', 'Nakajima'),
 (37, 'Kota', 'Nakanishi'),
 (38, 'Yoshiki', 'Yamashita'),
 (39, 'Teddie', 'Ande

In [122]:
conn.close()

In [115]:
print(thing)

(943,)


## Getting List of Cards

In [10]:
latest_event = events[0]
latest_event

'https://www.mtgtop8.com/event?e=28293&f=ST'

In [27]:
html = requests.get(latest_event)
soup = BeautifulSoup(html.text)
body = soup.body

for item in body.find_all("td", class_="G14"):
    result = item.text.strip().split(maxsplit=1)
    print(html.url)
    print(result + [item.find_all("span", class_="L14")[0].get("id")[:-2][5:]])

https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Cragcrown Pathway', '261']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Fabled Passage', '244']
https://www.mtgtop8.com/event?e=28293&f=ST
['3', 'Forest', '092']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Island', '130']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Ketria Triome', '250']
https://www.mtgtop8.com/event?e=28293&f=ST
['2', 'Mountain', '170']
https://www.mtgtop8.com/event?e=28293&f=ST
['2', 'Riverglide Pathway', '264']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Beanstalk Giant', '149']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Bonecrusher Giant', '115']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Brazen Borrower', '039']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Edgewall Innkeeper', '151']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Kazandu Mammoth', '189']
https://www.mtgtop8.com/event?e=28293&f=ST
['4', 'Lovestruck Beast', '299']
https://www.mtgtop8.com/event?e=28293&f=ST

In [52]:
conn = ""

Help on class SQLDatabase in module src.sqldb:

class SQLDatabase(builtins.object)
 |  SQLDatabase(db_name=None)
 |  
 |  API for interacting with SQL database.
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __init__(self, db_name=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add_new(self, item, table)
 |  
 |  close(self)
 |  
 |  get_dataframe_from(self, table)
 |  
 |  union_events(self, data, table)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [12]:
event_df

,id,name,link,date
0,1,FNM @ MTG Arena Campania,https://www.mtgtop8.com/event?e=27938&f=ST,2020-10-30
1,2,Torneios @ Ludo Quest,https://www.mtgtop8.com/event?e=27939&f=ST,2020-10-30
2,3,All Hallow's Eve FNM @ Gamer's Gauntlet,https://www.mtgtop8.com/event?e=27940&f=ST,2020-10-30
3,4,Japan Championship 2020 Autumn Last Chance Tri...,https://www.mtgtop8.com/event?e=27925&f=ST,2020-10-29
4,5,Free Daily @ Owl Central Games,https://www.mtgtop8.com/event?e=27918&f=ST,2020-10-28
...,...,...,...,...
360,361,Japan Championship 2020 Winter Daily Trial @ B...,https://www.mtgtop8.com/event?e=28286&f=ST,2020-11-25
361,362,Free Daily @ Owl Central Games,https://www.mtgtop8.com/event?e=28287&f=ST,2020-11-25
362,363,Keep it Simple Stupid @ Magic for Old Guys,https://www.mtgtop8.com/event?e=28288&f=ST,2020-11-25
363,364,Japan Championship 2020 Winter Daily Trial @ B...,https://www.mtgtop8.com/event?e=28293&f=ST,2020-11-26


In [18]:
thing = list("Title: " + event_df["name"])

In [19]:
thing

['Title: FNM @ MTG Arena Campania',
 'Title: Torneios @ Ludo Quest',
 "Title: All Hallow's Eve FNM @ Gamer's Gauntlet",
 'Title: Japan Championship 2020 Autumn Last Chance Trial @ BIG Magic',
 'Title: Free Daily @ Owl Central Games',
 'Title: Japan Championship 2020 Autumn Last Chance Trial @ BIG Magic',
 'Title: Monday Night Magic @ Plague League',
 'Title: Event @ Lotus eSports',
 'Title: Japan Championship 2020 Autumn - Last Chance Trial @ BIG Magic',
 'Title: Gentry ! @ The Gentry Magic League',
 'Title: mock Tournament Try 2 @ HavenGamesLLC',
 'Title: Tuesday Night Standard @ Misty Mountain Games',
 "Title: Cobram Cup - 5th Tappa @ Mind's Desire",
 'Title: Japan Championship 2020 Autumn - Last Chance Trial @ BIG Magic',
 'Title: Free Daily @ Owl Central Games',
 'Title: Weekly @ The Mythic Society',
 'Title: MTGO Standard Challenge',
 'Title: The 9th Hamazushi Tournament @ Hamakikaku (Toyama, Japan)',
 'Title: 6th Anniversary PWCS @ Hareruya (Japan)',
 'Title: Zendikar Rising Cham

In [43]:
thing = "1x"

In [40]:
int(thing)

1

In [44]:
assert thing.isdigit(), "Not a digit"

AssertionError: Not a digit